<a href="https://colab.research.google.com/github/ASi-F/NLPlay-with-Transformers/blob/main/GPT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install transformers

In [5]:
import nltk
import pandas as pd

In [7]:
df = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/articles.xlsx')

In [8]:
df['Articles'] = df['Articles'].apply(lambda x: x.replace('\n',''))

In [9]:
df['length'] = df['Articles'].apply(len)

In [10]:
df

,Articles,length
0,The dominant sequence transduction models are ...,2107
1,The goal of reducing sequential computation al...,2035
2,We introduce a new language representation mod...,2221
3,"In this paper, we improve the fine-tuning base...",2049
4,Many modern NLP systems rely on word embedding...,2597
5,We propose the Gaussian Error Linear Unit (GEL...,3425
6,This work presents a novel objective function ...,2165
7,Semantic Textual Similarity (STS) measures the...,2564
8,We present our UWB system for Semantic Textual...,2574
9,The recent tremendous success of unsupervised ...,2414


In [11]:
from transformers import GPT2LMHeadModel , GPT2Tokenizer

In [12]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2-large') 
model = GPT2LMHeadModel.from_pretrained('gpt2-large' , 
pad_token_id = tokenizer.eos_token_id)

In [13]:
df['ref'] = df['Articles'].apply(lambda x: tokenizer.decode(tokenizer.encode(x[:2000],return_tensors='pt')[0]))

In [ ]:
def gen(x):
  input_ids = tokenizer.encode(x[:200],return_tensors='pt')
  output = model.generate(input_ids, max_length = 2000, num_beams = 5,no_repeat_ngram_size  = 2,early_stopping = True)
  return output

df['gen'] = df['ref'].apply(gen)

/usr/local/lib/python3.7/dist-packages/torch/_tensor.py:575: UserWarning: floor_divide is deprecated, and will be removed in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values.
To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor'). (Triggered internally at  /pytorch/aten/src/ATen/native/BinaryOps.cpp:467.)
  return torch.floor_divide(self, other)


In [ ]:
from nltk.translate.bleu_score import SmoothingFunction, corpus_bleu, sentence_bleu


def bleu(ref, gen):
    ''' 
    calculate pair wise bleu score. uses nltk implementation
    Args:
        references : a list of reference sentences 
        candidates : a list of candidate(generated) sentences
    Returns:
        bleu score(float)
    '''
    ref_bleu = []
    gen_bleu = []
    for l in gen:
        gen_bleu.append(l.split())
    for i,l in enumerate(ref):
        ref_bleu.append([l.split()])
    cc = SmoothingFunction()
    score_bleu = corpus_bleu(ref_bleu, gen_bleu, weights=(0, 1, 0, 0), smoothing_function=cc.method4)
    return score_bleu

In [ ]:
x = bleu(list(df['ref']),list(df['gen'])